In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os

In [2]:
names2dirs = {'Baseline': 'baseline_nexp1_v1.6_10yrs_glance',
             'Baseline 2 snaps': 'baseline_nexp2_v1.6_10yrs_glance',
             'Barebones': 'barebones_v1.6_10yrs_glance',
             'DDF Heavy': 'ddf_heavy_v1.6_10yrs_glance',
             'DM Heavy': 'dm_heavy_v1.6_10yrs_glance',
             'MW Heavy': 'mw_heavy_v1.6_10yrs_glance',
              'Rolling Exgal': 'rolling_exgal_mod2_dust_sdf_0.80_v1.6_10yrs_glance',
              'SS Heavy': 'ss_heavy_v1.6_10yrs_glance',
               'Combo Dust': 'combo_dust_v1.6_10yrs_glance',}

In [3]:
root = 'sims_featureScheduler_runs1.6/potential_schedulers/'
conn = sqlite3.connect(root+names2dirs['Baseline']+'/resultsDb_sqlite.db')
filt2metricid = {'u': 99, 'g': 94, 'r': 84, 'i': 69, "z": 74, 'y': 79}


In [4]:

baseline_medians = []
for filtername in filt2metricid:
    const = "'" + 'filter="%s"' % filtername +"'"
    sql = 'select metricId from metrics where metricname="CoaddM5" and metricMetadata=" %s"' % filtername
    mid = np.max(pd.read_sql(sql, conn).values)
    sql = 'select summaryValue from summarystats where metricId=%i and summaryName="Median";' % mid
    baseline_medians.append(np.max(pd.read_sql(sql, conn).values))
conn.close()

In [5]:
diffs = []
for name in names2dirs:
    conn = sqlite3.connect(root+names2dirs[name]+'/resultsDb_sqlite.db')
    depths = []
    for filtername in filt2metricid:
        const = "'" + 'filter="%s"' % filtername +"'"
        sql = 'select metricId from metrics where metricname="CoaddM5" and metricMetadata=" %s"' % filtername
        mid = np.max(pd.read_sql(sql, conn).values)
        sql = 'select summaryValue from summarystats where metricId=%i and summaryName="Median";' % mid
        depths.append(np.max(pd.read_sql(sql, conn).values))
    diffs.append(np.array(baseline_medians) - np.array(depths))
    conn.close()

In [6]:
diffs

[array([0., 0., 0., 0., 0., 0.]),
 array([0.24062333, 0.11076862, 0.07816952, 0.07054639, 0.05744362,
        0.05662379]),
 array([-0.13033059, -0.14903502, -0.11523458, -0.17004907, -0.11926319,
        -0.13748785]),
 array([0.07706529, 0.08524253, 0.07730534, 0.10635514, 0.07655759,
        0.06285393]),
 array([ 0.11093656,  0.12250268,  0.06843433, -0.00520472, -0.01112527,
         0.03737058]),
 array([0.01891521, 0.00779402, 0.00554651, 0.014471  , 0.00992623,
        0.01030969]),
 array([0.11171172, 0.09777154, 0.1040644 , 0.10817452, 0.1057656 ,
        0.08796693]),
 array([-0.01602877,  0.0683776 ,  0.04687154,  0.1074376 ,  0.01786905,
         0.03405319]),
 array([0.11719467, 0.14191839, 0.13965777, 0.14524716, 0.10620789,
        0.08540689])]

In [7]:

# lets put it all into a dataframe
df = pd.DataFrame()
df['filter'] = list(filt2metricid.keys())

In [8]:
for i,key in enumerate(names2dirs):
    df[key] = diffs[i]

In [9]:
df['Baseline'] = baseline_medians

In [10]:
f = open('mag_diff.tex', 'w')
table = df.to_latex(float_format="%.2f", index=False)
print(table, file=f)
f.close()